In [ ]:
# Install Packages

# for path handling
import pathlib

# for loading images into numpy
import imageio

import numpy as np

# for greyscale conversion
from skimage.color import rgb2gray

# for plotting
import matplotlib.pyplot as plt

# Reading Data and Data Preparation

https://www.kaggle.com/code/vanausloos/how-to-read-images-in-python/notebook

In [ ]:
# get paths to images
paths_to_images = pathlib.Path('./images').glob('**/*.jpeg')
paths_sorted = sorted([x for x in paths_to_images])

sample_image = imageio.imread(paths_sorted[24])

In [ ]:
# initialize data set
train_X = np.array([imageio.imread(path) for path in paths_sorted])

# check if images are loaded propperly

def plot_image(image):
    plt.imshow(image,cmap="gray")
    plt.show()

plot_image(sample_image)
print("See image of colored bike? Nice! everything worked as expected.")

In [ ]:
# Grey Scale images
print(f"Colored (original) images shape: {sample_image.shape}")
# shows height, width and depth (which is colors r,g,b)

sample_image_grey = rgb2gray(sample_image)
print(f"Grey scale images shape: {sample_image_grey.shape}")
# colors have been removed

plot_image(sample_image_grey)
print("See black and white image of the same bike? Lets go!")

# Apply grey scale conversion to all images
train_X_grey = np.array([rgb2gray(image) for image in train_X])
